In [1]:
# imports
import time

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, desc, lit, rank, sum as _sum
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Start Spark Session
spark = SparkSession.builder \
    .appName("Query 2") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "4") \
    .getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1392,application_1732639283265_1354,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# paths
crime_data_2010_2019 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_data_2020 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

# Load datasets
crime_data = spark.read.csv(crime_data_2010_2019, header=True, inferSchema=True).union(
    spark.read.csv(crime_data_2020, header=True, inferSchema=True))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Start timing
start_time_df = time.time()

# Create a new column to identify closed cases 
closed_cases_df = crime_data.withColumn(
    "Is_Closed",
    when((col("Status Desc") != "UNK") & (col("Status Desc") != "Invest Cont"), lit(1)).otherwise(lit(0))
)

# Extract the year from the "Date Rptd" column
closed_cases_df = closed_cases_df.withColumn("Year", col("Date Rptd").substr(7, 4).alias("Year"))

# Group by precinct and year, calculate closure rates
closure_rate_df = closed_cases_df.groupBy("AREA NAME", "Year") \
    .agg(
        (count(when(col("Is_Closed") == 1, 1)) / count("*") * 100).alias("Closed_Case_Rate")
    )

# Define a window specification for ranking precincts by closure rate within each year
window_spec = Window.partitionBy("Year").orderBy(col("Closed_Case_Rate").desc())
ranked_closure_rate_df = closure_rate_df.withColumn("Rank", rank().over(window_spec))

# Filter for top 3 precincts per year and sort by year and rank
top_closure_rate_df = ranked_closure_rate_df.filter(col("Rank") <= 3).orderBy("Year", "Rank")

# Show the results
top_closure_rate_df.show(truncate=False, n=top_closure_rate_df.count())

# End timing
end_time_df = time.time()
elapsed_time_df = end_time_df - start_time_df
print(f"DataFrame Implementation Time: {elapsed_time_df:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+------------------+----+
|AREA NAME  |Year|Closed_Case_Rate  |Rank|
+-----------+----+------------------+----+
|Rampart    |2010|32.947355855318136|1   |
|Olympic    |2010|31.962706191728422|2   |
|Harbor     |2010|29.63203463203463 |3   |
|Olympic    |2011|35.212167689161554|1   |
|Rampart    |2011|32.511779630300836|2   |
|Harbor     |2011|28.652205202015008|3   |
|Olympic    |2012|34.414818310523835|1   |
|Rampart    |2012|32.9464181029429  |2   |
|Harbor     |2012|29.815133276010318|3   |
|Olympic    |2013|33.52812271731191 |1   |
|Rampart    |2013|32.08287360549221 |2   |
|Harbor     |2013|29.16422459266206 |3   |
|Van Nuys   |2014|31.80567315834039 |1   |
|West Valley|2014|31.31198995605775 |2   |
|Mission    |2014|31.16279069767442 |3   |
|Van Nuys   |2015|32.64134698172773 |1   |
|West Valley|2015|30.27597402597403 |2   |
|Mission    |2015|30.179460678380153|3   |
|Van Nuys   |2016|31.880755720117726|1   |
|West Valley|2016|31.54798761609907 |2   |
|Foothill  

In [11]:
# Register the DataFrame as a temporary SQL table
crime_data.createOrReplaceTempView("crime_data")

# Start timing
start_time_sql = time.time()

# SQL query to calculate closure rates, rank, and filter for top 3 per year
sql_query = """
        SELECT 
        Year,
        `AREA NAME` AS Precinct,
        Closed_Case_Rate,
        Rank
    FROM (
        SELECT
            substr(`Date Rptd`, 7, 4) AS Year,
            `AREA NAME`, 
            (COUNT(CASE WHEN `Status Desc` != 'UNK' AND `Status Desc` != 'Invest Cont' THEN 1 END) / COUNT(*) * 100) AS Closed_Case_Rate,
            RANK() OVER (
                PARTITION BY substr(`Date Rptd`, 7, 4)
                ORDER BY (COUNT(CASE WHEN `Status Desc` != 'UNK' AND `Status Desc` != 'Invest Cont' THEN 1 END) / COUNT(*) * 100) DESC
            ) AS Rank
        FROM crime_data
        GROUP BY
            `AREA NAME`,
            substr(`Date Rptd`, 7, 4)
    ) ranked_data
    WHERE Rank <= 3
    ORDER BY Year, Rank
"""

# Execute the SQL query
top_closure_rate_sql_df = spark.sql(sql_query)


# Show results
top_closure_rate_sql_df.show(truncate=False, n=top_closure_rate_df.count())

# End timing
end_time_sql = time.time()
elapsed_time_sql = end_time_sql - start_time_sql
print(f"SQL Implementation Time: {elapsed_time_sql:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+----+
|Year|Precinct   |Closed_Case_Rate  |Rank|
+----+-----------+------------------+----+
|2010|Rampart    |32.947355855318136|1   |
|2010|Olympic    |31.962706191728422|2   |
|2010|Harbor     |29.63203463203463 |3   |
|2011|Olympic    |35.212167689161554|1   |
|2011|Rampart    |32.511779630300836|2   |
|2011|Harbor     |28.652205202015008|3   |
|2012|Olympic    |34.414818310523835|1   |
|2012|Rampart    |32.9464181029429  |2   |
|2012|Harbor     |29.815133276010318|3   |
|2013|Olympic    |33.52812271731191 |1   |
|2013|Rampart    |32.08287360549221 |2   |
|2013|Harbor     |29.16422459266206 |3   |
|2014|Van Nuys   |31.80567315834039 |1   |
|2014|West Valley|31.31198995605775 |2   |
|2014|Mission    |31.16279069767442 |3   |
|2015|Van Nuys   |32.64134698172773 |1   |
|2015|West Valley|30.27597402597403 |2   |
|2015|Mission    |30.179460678380153|3   |
|2016|Van Nuys   |31.880755720117726|1   |
|2016|West Valley|31.54798761609907 |2   |
|2016|Footh

In [12]:
group_number = "11"
s3_path = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/parquet/"

crime_data_parquet = crime_data

# Save the dataset as a single Parquet file in the specified S3 bucket
crime_data_parquet.write.mode("overwrite").parquet(s3_path)
print("Crime dataset saved as Parquet.")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Crime dataset saved as Parquet.

In [13]:
# Load data from CSV
start_time_csv = time.time()
crime_csv_df = spark.read.csv(crime_data_2010_2019, header=True, inferSchema=True).union(
    spark.read.csv(crime_data_2020, header=True, inferSchema=True))
# crime_csv_df.show(1)
end_time_csv = time.time()
elapsed_time_csv = end_time_csv - start_time_csv
print(f"CSV Load Time: {elapsed_time_csv:.2f} seconds")

# Load data from Parquet
start_time_parquet = time.time()
crime_parquet_df = spark.read.parquet(s3_path)
# crime_parquet_df.show(1)
end_time_parquet = time.time()
elapsed_time_parquet = end_time_parquet - start_time_parquet
print(f"Parquet Load Time: {elapsed_time_parquet:.2f} seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CSV Load Time: 3.95 seconds
Parquet Load Time: 0.33 seconds

In [14]:
# Start timing
start_time_df_parq = time.time()

# Create a new column to identify closed cases 
closed_cases_df_parq = crime_parquet_df.withColumn(
    "Is_Closed",
    when((col("Status Desc") != "UNK") & (col("Status Desc") != "Invest Cont"), lit(1)).otherwise(lit(0))
)
# Extract the year from the "Date Rptd" column
closed_cases_df_parq = closed_cases_df_parq.withColumn("Year", col("Date Rptd").substr(7, 4).alias("Year"))

# Group by precinct and year, calculate closure rates
closure_rate_df_parq = closed_cases_df_parq.groupBy("AREA NAME", "Year") \
    .agg(
        (count(when(col("Is_Closed") == 1, 1)) / count("*") * 100).alias("Closed_Case_Rate")
    )

# Define a window specification for ranking precincts by closure rate within each year
window_spec = Window.partitionBy("Year").orderBy(col("Closed_Case_Rate").desc())
ranked_closure_rate_df_parq = closure_rate_df_parq.withColumn("Rank", rank().over(window_spec))

# Filter for top 3 precincts per year and sort by year and rank
top_closure_rate_df_parq = ranked_closure_rate_df_parq.filter(col("Rank") <= 3).orderBy("Year", "Rank")

# Show the results
top_closure_rate_df_parq.show(truncate=False, n=top_closure_rate_df.count())

# End timing
end_time_df_parq = time.time()
elapsed_time_df_parq = end_time_df_parq - start_time_df_parq
print(f"DataFrame (Parquet) Implementation Time: {elapsed_time_df_parq:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+------------------+----+
|AREA NAME  |Year|Closed_Case_Rate  |Rank|
+-----------+----+------------------+----+
|Rampart    |2010|32.947355855318136|1   |
|Olympic    |2010|31.962706191728422|2   |
|Harbor     |2010|29.63203463203463 |3   |
|Olympic    |2011|35.212167689161554|1   |
|Rampart    |2011|32.511779630300836|2   |
|Harbor     |2011|28.652205202015008|3   |
|Olympic    |2012|34.414818310523835|1   |
|Rampart    |2012|32.9464181029429  |2   |
|Harbor     |2012|29.815133276010318|3   |
|Olympic    |2013|33.52812271731191 |1   |
|Rampart    |2013|32.08287360549221 |2   |
|Harbor     |2013|29.16422459266206 |3   |
|Van Nuys   |2014|31.80567315834039 |1   |
|West Valley|2014|31.31198995605775 |2   |
|Mission    |2014|31.16279069767442 |3   |
|Van Nuys   |2015|32.64134698172773 |1   |
|West Valley|2015|30.27597402597403 |2   |
|Mission    |2015|30.179460678380153|3   |
|Van Nuys   |2016|31.880755720117726|1   |
|West Valley|2016|31.54798761609907 |2   |
|Foothill  